In [1]:
using CSV,DataFrames,Mosek, MosekTools,JuMP,Dates,Random


function SNP(file,numCompanies, allocationAmount,symbol)
    # numCompanies is the number of companies that is taken into account to compare with the index. Ensure
    # that this is less than or equal to the number of companies in the dataset. For example, if the number 
    # of companies is 100 then the stock allocatin provided closely matches the S&P 500 index.
    
    # alllocationAmount is the money that the investor is putting into numCompanies
    model = Model(with_optimizer(Mosek.Optimizer))
    set_silent(model)
    data = CSV.read(file, DataFrame)
    data = data[1:numCompanies,:]
    mktCapital= data[:,"MarketCapitalization(%)"]/sum(data[:,"MarketCapitalization(%)"])
    stockStr = "StockPrice("*symbol*")"
    stockPrice=data[:,stockStr]
    
    @variable(model, 0<=stockAllocation[1:numCompanies],Int) # stocks for each company
    @variable(model, 0<=disSq[1:numCompanies])
    for i=1:numCompanies
        @constraint(model, (stockPrice[i]*stockAllocation[i] - allocationAmount*mktCapital[i])^2 <=disSq[i])
    end
    @objective(model, Min, 1/numCompanies*sum(disSq[i] for i=1:numCompanies))
    optimize!(model)
    data[!,"Allocations"] = value.(stockAllocation)
    println("Difference from the S&P",string(numCompanies)," index: ",
        symbol, string(round(sum(stockPrice[i]*data[i,"Allocations"]
                    -allocationAmount*mktCapital[i] for i=1:numCompanies))))
    println("Total money allocated in stocks: "*symbol,
        round(sum(stockPrice[i]*data[i,"Allocations"] for i=1:numCompanies)))
    println("RMSE:",sqrt(objective_value(model)))
    return data
end


function AllocationDifference(prevFile, newFile,symbo)
    prevData = CSV.read(prevFile,DataFrame)
    newData = CSV.read(newFile,DataFrame)
    
    newDataFrame=DataFrame(CompanyName=[],Symbol=[])
    newDataFrame."Prev Allocation"=[]
    newDataFrame."New Allocation"=[]
    newDataFrame."Change in Allocation"=[]
    
    # considers all companies common between the 2 dataframes and only those present in the prevData
    for i in 1:nrow(prevData)
        if prevData[i,"Symbol"] in newData."Symbol"
            j = findIndex(newData."Symbol",prevData[i,"Symbol"])
                push!(newDataFrame,[newData[j,"CompanyName"],
                                    newData[j,"Symbol"],
                                    prevData[i,"Allocations"],
                                    newData[j,"Allocations"],
                                    newData[j,"Allocations"]-prevData[i,"Allocations"]
                        ])
        else
                push!(newDataFrame,[prevData[i,"CompanyName"],
                                    prevData[i,"Symbol"],
                                    prevData[i,"Allocations"],
                                    "--",
                                    "--"
                        ])
        end
    end
     # considers only those companies present in newData
    for i in 1:nrow(newData)
        if newData[i,"Symbol"] ∉ prevData."Symbol"
            push!(newDataFrame,[newData[i,"CompanyName"],
                                newData[i,"Symbol"],
                                "--",
                                newData[i,"Allocations"],
                                newData[i,"Allocations"]
                        ])
        end
    end
    return newDataFrame
end

function findIndex(list,value)
    outInd = -1
    for (ind, val) in enumerate(list)
        if val==value
            outInd = ind
        end
    end
    return outInd
end
function SaveComparative(dataMatrix, numCompanies,allocationAmonunt,symbol)
    filename = "Results/Comparative/"*symbol*string(numCompanies)*"_"*string(allocationAmount)
    filename *= "_"*string(today())*".csv"
    CSV.write(filename, dataMatrix)
end

function SaveCapitalizations(dataMatrix, numCompanies,allocationAmount,symbol)
    filename = "Results/CapitalizationAndStockPrice/"*symbol*string(numCompanies)
    filename *= "_"*string(allocationAmount)*"_"*string(today())*".csv"
    CSV.write(filename, dataMatrix)
end

SaveCapitalizations (generic function with 1 method)

In [16]:
symbol="\$"
numCompanies=30
allocationAmount=31750

# file = "Data/S&P 500 allocations.csv"
# dataMatrix = SNP(file,numCompanies,allocationAmount,symbol)
# print(dataMatrix)
# SaveCapitalizations(dataMatrix,numCompanies,allocationAmount,symbol)
    

newFile = "Results/CapitalizationAndStockPrice/\$30_31750_2024-03-30.csv"
prevFile= "Results/CapitalizationAndStockPrice/\$40_38851_2024-02-28.csv"
dataMatrix=AllocationDifference(prevFile,newFile,symbol)
print(dataMatrix)
# SaveComparative(dataMatrix, numCompanies,allocationAmount,symbol)



40×5 DataFrame
 Row │ CompanyName                        Symbol  Prev Allocation  New Allocation  Change in Allocation 
     │ Any                                Any     Any              Any             Any                  
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Microsoft Corp                     MSFT    13.0             14.0            1.0
   2 │ Apple Inc.                         AAPL    26.0             --              --
   3 │ Nvidia Corp                        NVDA    4.0              --              --
   4 │ Amazon.com Inc                     AMZN    16.0             18.0            2.0
   5 │ Meta Platforms, Inc. Class A       META    4.0              4.0             0.0
   6 │ Alphabet Inc. Class A              GOOGL   11.0             11.0            0.0
   7 │ Berkshire Hathaway Class B         BRK.B   3.0              3.0             0.0
   8 │ Alphabet Inc. Class C              GOOG    9.0          

In [19]:
# using Dates,CSV,DataFrames
# function StockCalculator(oldFile,newFile)
#     oldData = CSV.read(oldFile,DataFrame)
#     BuyingDates = oldData[:,"Buying Date (YYYY-MM-DD)"]
#     println(BuyingDates[2]-today())
#     return oldData
# end
# oldFile = "./Data/S&P Prev.csv"
# newFile = "./Data/S&P 500 allocations.csv"
# print(StockCalculator(oldFile,newFile))
